In [1]:
!pip install konlpy
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 37.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ✅ 프로젝트 경로 설정
import os

project_path = "/content/drive/MyDrive/Colab Notebooks/youtube/youtube_interest_classifier"
os.makedirs(project_path, exist_ok=True)
os.chdir(project_path)

In [4]:
pwd

'/content/drive/MyDrive/Colab Notebooks/youtube/youtube_interest_classifier'

In [14]:
# csv_path='./data'

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import wandb

In [6]:
# 1f963dddeff10490dccf580f75dbc47d62af96f7
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kiipo0623 (kiipo0623-daejeon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# ✅ 샘플 학습 데이터셋
# train_data = pd.DataFrame({
#     "creator": ["손흥민채널", "쿡잇", "손흥민채널", "김장마스터", "경제왕", "요리연구소"],
#     "title": [
#         "손흥민 골 모음", "김장하는 방법", "토트넘 경기 리뷰",
#         "김장 레시피 공개", "금리 상승 영향 분석", "된장찌개 끓이는 법"
#     ],
#     "category": ["스포츠", "요리", "스포츠", "요리", "경제", "요리"]
# })

In [7]:
pwd

'/content/drive/MyDrive/Colab Notebooks/youtube/youtube_interest_classifier'

In [8]:
df1 = pd.read_csv("./data/youtube_fashion_data.csv")
df2 = pd.read_csv("./data/youtube_politics_data.csv")

# ✅ 합치고 섞기
df = pd.concat([df1, df2], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

assert 'channel_name' in df.columns and 'video_title' in df.columns and 'category' in df.columns
train_data = df

In [9]:
# ✅ 형태소 분석 기반 입력 텍스트 생성
def combine_creator_title(creator, title):
    return f"YouTuber: {creator} [SEP] Title: {title}"

train_data['input_text'] = train_data.apply(
    lambda row: combine_creator_title(row['channel_name'], row['video_title']),
    axis=1
)

In [10]:
# ✅ 라벨 인코딩
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['category'])

In [11]:
# ✅ HuggingFace Dataset 변환 및 토크나이징
model_name = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = Dataset.from_pandas(train_data[['input_text', 'label']])

def tokenize_function(example):
    return tokenizer(example["input_text"], padding="max_length", truncation=True, max_length=64)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
split_dataset = tokenized_datasets.train_test_split(test_size=0.2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/2036 [00:00<?, ? examples/s]

In [12]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

In [13]:
from transformers import EarlyStoppingCallback

# ✅ 모델 및 Trainer 설정
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_))
training_args = TrainingArguments(
    output_dir="./model_output",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss", # 기준 메트릭
    greater_is_better=False,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,
    logging_dir="./logs",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    compute_metrics=compute_metrics
)

# ✅ 학습 수행
trainer.train()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-4401e3d814d1>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.909900,0.769635,0.860294
2,0.366400,0.410140,0.931373
3,0.488500,0.357471,0.943627
4,0.216000,0.343232,0.950980
5,0.182400,0.329592,0.950980


TrainOutput(global_step=1020, training_loss=0.5500412616659613, metrics={'train_runtime': 198.2576, 'train_samples_per_second': 41.058, 'train_steps_per_second': 5.145, 'total_flos': 267744343342080.0, 'train_loss': 0.5500412616659613, 'epoch': 5.0})

In [16]:
# ✅ 모델 및 라벨 저장
model.save_pretrained(os.path.join(project_path, "model"))
tokenizer.save_pretrained(os.path.join(project_path, "model"))
import joblib
joblib.dump(le, os.path.join(project_path, "label_encoder.pkl"))

['/content/drive/MyDrive/Colab Notebooks/youtube/youtube_interest_classifier/label_encoder.pkl']

In [15]:
print(le.classes_)

['Autos , Vehicles' 'Comedy' 'Education' 'Entertainment'
 'Film , Animation' 'Gaming' 'Howto , Style' 'Music' 'News , Politics'
 'Nonprofits , Activism' 'People , Blogs' 'Pets , Animals' 'Sports'
 'Travel , Events']
